# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [6]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ludovicagonella/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ludovicagonella/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [8]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [9]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [10]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [11]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/ludovicagonella/code/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/ludovicagonella/code/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/ludovicagonella/code/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [12]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [13]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '6d5c66'. Skipping!
Property 'summary' already exists in node 'ba0faa'. Skipping!
Property 'summary' already exists in node '88c0ec'. Skipping!
Property 'summary' already exists in node '094768'. Skipping!
Property 'summary' already exists in node '9049aa'. Skipping!
Property 'summary' already exists in node 'e45a35'. Skipping!
Property 'summary' already exists in node '560a80'. Skipping!
Property 'summary' already exists in node '6ba8cc'. Skipping!
Property 'summary' already exists in node 'de7aaa'. Skipping!
Property 'summary' already exists in node '77de2f'. Skipping!
Property 'summary' already exists in node '90987e'. Skipping!
Property 'summary' already exists in node '4c2fba'. Skipping!
Property 'summary' already exists in node '5c1f48'. Skipping!
Property 'summary' already exists in node '06bc0c'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '6d5c66'. Skipping!
Property 'summary_embedding' already exists in node 'ba0faa'. Skipping!
Property 'summary_embedding' already exists in node '6ba8cc'. Skipping!
Property 'summary_embedding' already exists in node 'e45a35'. Skipping!
Property 'summary_embedding' already exists in node 'de7aaa'. Skipping!
Property 'summary_embedding' already exists in node '88c0ec'. Skipping!
Property 'summary_embedding' already exists in node '9049aa'. Skipping!
Property 'summary_embedding' already exists in node '77de2f'. Skipping!
Property 'summary_embedding' already exists in node '094768'. Skipping!
Property 'summary_embedding' already exists in node '90987e'. Skipping!
Property 'summary_embedding' already exists in node '4c2fba'. Skipping!
Property 'summary_embedding' already exists in node '560a80'. Skipping!
Property 'summary_embedding' already exists in node '06bc0c'. Skipping!
Property 'summary_embedding' already exists in node '5c1f48'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [18]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

- single-hop: you can get the needed information with one single query. The number is the percentage of the total queries that should be generated using this method
- multi-hop abstract synth: will generate queries that require to get context from multiple sources in order to generate the answer. These questions require reasoning
- multi-hop specific: generate answers that are grounded in one single document but require more context



Finally, we can use our `TestSetGenerator` to generate our testset!

In [19]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of the School Participation D...,"[Chapter 1 Academic Years, Academic Calendars,...",The School Participation Division oversees the...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(b) say about weeks of i...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,Whaat is Chapter 3 in the contex of the guidel...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term ...,single_hop_specifc_query_synthesizer
3,Is the Federal Work-Study (FWS) program subjec...,[Non-Term Characteristics A program that measu...,"No, the Federal Work-Study (FWS) program is an...",single_hop_specifc_query_synthesizer
4,What is the significance of the Direct Loan in...,[both the credit or clock hours and the weeks ...,The Direct Loan program is affected by the cre...,single_hop_specifc_query_synthesizer
5,How does including clinical work outside stand...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Including clinical work conducted outside stan...,multi_hop_abstract_query_synthesizer
6,how academic year based on instructional time ...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that each eligible progra...,multi_hop_abstract_query_synthesizer
7,How does credit hour allocation for clinical e...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Inclusion of clinical work in a standard term ...,multi_hop_abstract_query_synthesizer
8,Chapter 2 include clinical work in standard te...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Inclusion of clinical work in a standard term ...,multi_hop_specific_query_synthesizer
9,How do Appendix A and Appendix B provide guida...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix B offers detailed guidance on disburs...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [20]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'c1c648'. Skipping!
Property 'summary' already exists in node 'f368be'. Skipping!
Property 'summary' already exists in node '527b69'. Skipping!
Property 'summary' already exists in node '45dd67'. Skipping!
Property 'summary' already exists in node '02fb5b'. Skipping!
Property 'summary' already exists in node '23ec5a'. Skipping!
Property 'summary' already exists in node '1b3d3b'. Skipping!
Property 'summary' already exists in node 'a64381'. Skipping!
Property 'summary' already exists in node '9e45d0'. Skipping!
Property 'summary' already exists in node '3a5236'. Skipping!
Property 'summary' already exists in node 'c9fced'. Skipping!
Property 'summary' already exists in node 'f8aaac'. Skipping!
Property 'summary' already exists in node '7b9045'. Skipping!
Property 'summary' already exists in node 'd9392a'. Skipping!


...

Property 'summary_embedding' already exists in node 'c1c648'. Skipping!
Property 'summary_embedding' already exists in node '527b69'. Skipping!
Property 'summary_embedding' already exists in node 'a64381'. Skipping!
Property 'summary_embedding' already exists in node '45dd67'. Skipping!
Property 'summary_embedding' already exists in node '9e45d0'. Skipping!
Property 'summary_embedding' already exists in node '02fb5b'. Skipping!
Property 'summary_embedding' already exists in node 'f368be'. Skipping!
Property 'summary_embedding' already exists in node '23ec5a'. Skipping!
Property 'summary_embedding' already exists in node '3a5236'. Skipping!
Property 'summary_embedding' already exists in node '1b3d3b'. Skipping!
Property 'summary_embedding' already exists in node 'c9fced'. Skipping!
Property 'summary_embedding' already exists in node 'd9392a'. Skipping!
Property 'summary_embedding' already exists in node 'f8aaac'. Skipping!
Property 'summary_embedding' already exists in node '7b9045'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [22]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Knowledge Center?,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not provide a specific defini...,single_hop_specifc_query_synthesizer
1,What is the regulation 34 CFR 668.3(b) about i...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What does Volume 8 specify regarding clinical ...,[Inclusion of Clinical Work in a Standard Term...,Volume 8 states that clinical work conducted o...,single_hop_specifc_query_synthesizer
3,What are Non-Term Characteristics in relation ...,[Non-Term Characteristics A program that measu...,A program that measures progress in credit-hou...,single_hop_specifc_query_synthesizer
4,If student not finish enough hours or weeks in...,[<1-hop>\n\nboth the credit or clock hours and...,"Based on the context, disbursement of federal ...",multi_hop_abstract_query_synthesizer
5,Weeks of instructional time 34 CFR 668.3(b) ho...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...","According to 34 CFR 668.3(b), the academic yea...",multi_hop_abstract_query_synthesizer
6,In the context of disbursement timing in subsc...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that in subscription-base...,multi_hop_abstract_query_synthesizer
7,what is the academic year for Title IV purpose...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The academic year for Title IV purposes is def...,multi_hop_abstract_query_synthesizer
8,Wha Volume 2 and Volume 8 contans about academ...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Volume 2 discusses the requirements for defini...,multi_hop_specific_query_synthesizer
9,How do the definitions of academic years and i...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 2 explains that the disbursement of fed...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [23]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [24]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [25]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [27]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [28]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [29]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [30]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [32]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [33]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'Based on the provided context, the kinds of loans available include:\n\n- Direct Subsidized Loans (available only to undergraduate students)\n- Direct Unsubsidized Loans\n- Direct PLUS Loans (also called student Federal PLUS Loans or parent Direct PLUS Loans)\n- Subsidized and Unsubsidized Federal Stafford Loans (originally under the FFEL Program, but no new loans made after July 1, 2010)\n- Federal SLS Loans (under the FFEL Program, no new loans after July 1, 2010)\n\nThese loans are made under the Direct Loan Program or were previously made under the FFEL Program before it ended.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [34]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [35]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: evaluate predictions relative to their inputs and/or reference labels. This is a hard truth evaluator based on labels
- `labeled_helpfulness_evaluator`: evaluates is the answer is useful to the user
- `empathy_evaluator`: evaluates if the response is empathic and makes the user feel heard

## LangSmith Evaluation

In [36]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'large-dinner-84' at:
https://smith.langchain.com/o/6ce7a019-3815-4374-be47-38dfdf42ee54/datasets/ecd9d940-b0a1-43ba-97fb-267f6dabad2e/compare?selectedSessions=f0eebbd3-6f0f-4af1-8145-4f1b0db2a0eb




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Volume 8 and Volume 7 relate to the inc...,Based on the provided context:\n\nVolume 8 dis...,None,Volume 8 discusses the inclusion of clinical w...,1,1,0,14.765921,d89db3ce-c22c-4f23-9fb0-fc7508e4e1ee,4fb2617f-3e03-4bac-af9e-723f75bba94e
1,How do Appendix A and Appendix B relate to dis...,I don't know.,None,Appendix A provides guidance on calculating Pe...,0,0,0,0.743288,cc7701a5-4324-4d73-a301-1d95b6630757,ce01f9d6-7cd1-4485-985d-6c3a51f3ec5a
2,How do the definitions of academic years and i...,Based on the provided context:\n\nThe definiti...,None,Volume 2 explains that the disbursement of fed...,1,1,0,5.891465,c891d807-3b7b-4755-bdb7-af605f6e1bc1,90d3201b-dc09-4644-ad23-d2a57f6c3fac
3,Wha Volume 2 and Volume 8 contans about academ...,"Based on the provided context, Volume 2, Chapt...",None,Volume 2 discusses the requirements for defini...,1,0,0,5.036380,31693c51-fec8-42ab-9925-2f9654239cb4,7a0715a1-56eb-49d8-ad93-5ec594afcf6a
4,what is the academic year for Title IV purpose...,The academic year for Title IV purposes is def...,None,The academic year for Title IV purposes is def...,1,1,0,5.599937,f47b0f55-7a09-49d4-bf85-e6bc2425b540,712d9ef8-c92a-4dd1-bb45-e0220bd492d2
5,In the context of disbursement timing in subsc...,"In subscription-based programs, for the first ...",None,The context explains that in subscription-base...,1,1,0,5.525159,65b947b6-cdf5-47af-98ee-061960d93ad1,29349ae2-e83d-40cd-b2e6-ee9011c51468
6,Weeks of instructional time 34 CFR 668.3(b) ho...,"Based on the context, for an academic year the...",None,"According to 34 CFR 668.3(b), the academic yea...",1,1,0,2.890696,f9b9c3fc-5142-4835-bdb0-efa4e3258314,799edfe7-efa6-4575-b5f3-e56011ee7cf4
7,If student not finish enough hours or weeks in...,Based on the provided context:\n\n1. **Effect ...,None,"Based on the context, disbursement of federal ...",1,1,0,14.881108,9a71760b-97f4-4fba-b6c9-643113cd4795,671401fa-eedd-4579-8d20-70f31060928b
8,What are Non-Term Characteristics in relation ...,Non-Term Characteristics in relation to credit...,None,A program that measures progress in credit-hou...,1,1,0,2.660066,f224bed3-357c-4024-9a3e-443d90b783a5,72142477-3b74-4780-add6-7e6bcccc089f
9,What does Volume 8 specify regarding clinical ...,"Volume 8, Chapter 3 provides additional guidan...",None,Volume 8 states that clinical work conducted o...,0,0,0,1.874913,090dfb3e-bcf5-4886-8f1a-0a70911a192d,4f536636-4bdc-45bc-a8b1-ab79c1f522fd


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [39]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [40]:
rag_documents = docs

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

Increasing the chunk size could help capture more rich context based on multiple paragraphs/ sentences or it could also merge multiple contexts together and ending up confusing the llm. It all depends on the data we have and how it is structured.

In [42]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application? 

The embedding define how chunks are represented as vector. Bigger or smaller embedding spaces might mean more or less compression of information. Moreover how the embedder model was trained will affect how it transforms the chunks and how effectively clusters topics that are similar in the specific space of the application together.

In [43]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [44]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [45]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [46]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question. Based on the information you've shared, there are several types of federal student loans available to help with college costs:\n\n1. **Direct Subsidized Loans** – These are loans for students with demonstrated financial need, where the government pays the interest while the student is in school at least half-time. There is a maximum annual loan amount based on the student's grade level and dependency status.\n\n2. **Direct Unsubsidized Loans** – These loans are available to both dependent and independent students regardless of financial need. Interest accrues while the student is in school.\n\n3. **Direct PLUS Loans** – These are loans for parents of dependent students or for graduate/professional students themselves, which can help cover the cost of attendance not met by other financial aid. There is no fixed loan limit, but the amount cannot exceed the student's total cost of attendance minus other financial aid.\n\nIt’s also mentioned that if a parent i

Finally, we can evaluate the new chain on the same test set!

In [47]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'upbeat-war-29' at:
https://smith.langchain.com/o/6ce7a019-3815-4374-be47-38dfdf42ee54/datasets/ecd9d940-b0a1-43ba-97fb-267f6dabad2e/compare?selectedSessions=f17113b4-531e-40fb-9e4b-031d3f98d971




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Volume 8 and Volume 7 relate to the inc...,Thank you for your thoughtful question. From t...,None,Volume 8 discusses the inclusion of clinical w...,1,0,1,6.116048,d89db3ce-c22c-4f23-9fb0-fc7508e4e1ee,562c450d-9c82-4682-a241-c8adf6dcb5c3
1,How do Appendix A and Appendix B relate to dis...,Thank you for your thoughtful question. Based ...,None,Appendix A provides guidance on calculating Pe...,1,1,1,4.680564,cc7701a5-4324-4d73-a301-1d95b6630757,4159449a-81e8-43f3-8f30-4755b116c669
2,How do the definitions of academic years and i...,Thank you for your thoughtful question. I can ...,None,Volume 2 explains that the disbursement of fed...,1,0,1,13.069852,c891d807-3b7b-4755-bdb7-af605f6e1bc1,c217cbfe-4739-49c6-99df-66a30076c1d1
3,Wha Volume 2 and Volume 8 contans about academ...,Thank you for your thoughtful question. Based ...,None,Volume 2 discusses the requirements for defini...,1,1,1,5.705077,31693c51-fec8-42ab-9925-2f9654239cb4,67a5cea0-c1a5-412e-ac6a-bc660ac5d645
4,what is the academic year for Title IV purpose...,Thank you for your thoughtful question. Based ...,None,The academic year for Title IV purposes is def...,1,0,1,5.598255,f47b0f55-7a09-49d4-bf85-e6bc2425b540,f1cb2342-c3fc-45e6-9d1e-00a0a56a5fb3
5,In the context of disbursement timing in subsc...,Thank you for your thoughtful question—it's im...,None,The context explains that in subscription-base...,1,1,1,6.037770,65b947b6-cdf5-47af-98ee-061960d93ad1,231607ac-d8d5-4aa7-bd3d-26ba6a5f2db2
6,Weeks of instructional time 34 CFR 668.3(b) ho...,Thank you for your question. Based on the cont...,None,"According to 34 CFR 668.3(b), the academic yea...",1,1,1,3.298613,f9b9c3fc-5142-4835-bdb0-efa4e3258314,21a119ef-23e0-4666-b2f1-c1e28f7c101c
7,If student not finish enough hours or weeks in...,Thank you for your thoughtful question. It sou...,None,"Based on the context, disbursement of federal ...",1,1,1,13.416241,9a71760b-97f4-4fba-b6c9-643113cd4795,c056703b-5d5e-4793-8446-7b4325169dd8
8,What are Non-Term Characteristics in relation ...,Thank you for your thoughtful question! Based ...,None,A program that measures progress in credit-hou...,1,1,1,4.159827,f224bed3-357c-4024-9a3e-443d90b783a5,134d9298-6f31-441e-8d9d-22b085545238
9,What does Volume 8 specify regarding clinical ...,Thank you for your thoughtful question. Based ...,None,Volume 8 states that clinical work conducted o...,1,0,1,3.638675,090dfb3e-bcf5-4886-8f1a-0a70911a192d,677c0141-7e27-4be2-8e3f-5e3b1ce4d4a9


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.


Here is the metric results from the base run
![Base run](images/base_run.png)


Here is the metric results from the run with the tweaked prompt
![Empathy run](images/empathy_run.png)

The second run performs better on the empathy metric because the LLM was instructed to care about this aspect. Interestingly the second run uses way less tokens despite having the longer context window. It is in general less helpful which I suspect it has to do with bad context windows: when looking at the single answers it looks like they are a blurb of empathy + a synthesis of the context without real reasoning